In [1]:
import pandas as pd
import joblib
import os

In [2]:
file_path = os.path.dirname((os.path.dirname(os.getcwd()))) + '/predict_next_years_new_requests_distribution/models/sir/predict_total_requests_api.ipynb'
file_path

'/Users/sararozin/Documents/tama_project/Tama/predict_next_years_approved_requests_distribution/predict_next_years_new_requests_distribution/models/sir/predict_total_requests_api.ipynb'

In [3]:
%run $file_path

In [4]:
tama_df = pd.read_csv('engineered_tama_data.csv')
tama_df

,hebrew_city_name,city_code,year,total_open_requests,total_requests,total_approved_requests,longitude,latitude
0,אור יהודה,2400,2005,0,0.0,0.0,34.852394,32.030971
1,אור יהודה,2400,2006,0,0.0,0.0,34.852394,32.030971
2,אור יהודה,2400,2007,0,0.0,0.0,34.852394,32.030971
3,אור יהודה,2400,2008,0,1.0,0.0,34.852394,32.030971
4,אור יהודה,2400,2009,1,0.0,0.0,34.852394,32.030971
...,...,...,...,...,...,...,...,...
1075,תל אביב יפו,5000,2018,687,164.0,83.0,34.781806,32.085300
1076,תל אביב יפו,5000,2019,768,116.0,149.0,34.781806,32.085300
1077,תל אביב יפו,5000,2020,735,238.0,115.0,34.781806,32.085300
1078,תל אביב יפו,5000,2021,858,240.0,124.0,34.781806,32.085300


In [5]:
from datetime import datetime

def guess_total_open_requests_futured_year(city, futured_year):
    current_year = datetime.now().year
    max_year = 2022
    current_year = max_year
    print('current_year', current_year)
    rf_regressor = joblib.load('random_forest_model.pkl')
    tama_df = pd.read_csv('engineered_tama_data.csv')
    total_open_requests = tama_df[(tama_df['city_code'] == city) & (tama_df['year'] == min(current_year, futured_year))]['total_open_requests']
    if total_open_requests.empty:
        total_open_requests = 0
    else:
        total_open_requests = total_open_requests.iloc[0]
    if futured_year > current_year:
        for year in range(current_year, futured_year):
            print('--------------', year, '-----------------')
            print('total_open_requests: ', total_open_requests)
            predict_total_approved_requests = rf_regressor.predict([[city, year, total_open_requests]])[0]
            print('predict_total_approved_requests:', predict_total_approved_requests)
            predict_total_new_requests = predict_total_requests(city, year)
            print('predict_total_new_requests:', predict_total_new_requests)
            total_open_requests = total_open_requests - predict_total_approved_requests + predict_total_new_requests
    return total_open_requests

In [8]:
def predict_total_approved_requests(city, futured_year):
    guess_total_open_requests = guess_total_open_requests_futured_year(city, futured_year)
    rf_regressor = joblib.load('random_forest_model.pkl')
    total_approved_requests = rf_regressor.predict([[city, futured_year, guess_total_open_requests]])[0]
    if total_approved_requests > guess_total_open_requests:
        return guess_total_open_requests
    return total_approved_requests

### example code

In [9]:
predict_total_approved_requests(3000, 2027)

current_year 2022
-------------- 2022 -----------------
total_open_requests:  274
predict_total_approved_requests: 53.7
predict_total_new_requests: 58.6142207277618
-------------- 2023 -----------------
total_open_requests:  278.9142207277618
predict_total_approved_requests: 53.7
predict_total_new_requests: 46.54935433078496
-------------- 2024 -----------------
total_open_requests:  271.7635750585468
predict_total_approved_requests: 52.8
predict_total_new_requests: 35.46014509412466
-------------- 2025 -----------------
total_open_requests:  254.42372015267145
predict_total_approved_requests: 52.8
predict_total_new_requests: 26.178586070227887
-------------- 2026 -----------------
total_open_requests:  227.80230622289935
predict_total_approved_requests: 47.7
predict_total_new_requests: 18.887721225510404


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature 

47.7